In [1]:
import pynwb
from allensdk.brain_observatory.ecephys.dynamic_gating_ecephys_session import DynamicGatingEcephysSession
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline

import xarray as xr
import sys

# Add the path you want to include
sys.path.append('code')

from shield_utils import find_animals, get_lfp_dict, downsample, align_lfp

import json

data_path = '/oscar/data/sjones/kduecker/shield_data/'

meta_path = 'externals/SHIELD_Dynamic_Gating_Analysis'


In [ ]:
down_srate = 500
roi = ['LGd', 'VISp', 'VISl']
toi = [0, 2]

In [3]:
# load animals of interest

with open('session_info.json', 'r') as file:
    session_info = json.load(file)


# reorganize a bit for ease
mice_sess = dict()

for session in session_info['sessions_all_areas']:
    mice_sess[session[0:6]] = []

for session in session_info['sessions_two_areas']:
    mice_sess[session[0:6]] = []


for session in session_info['sessions_all_areas']:
    mice_sess[session[0:6]].append(session[8:])

for session in session_info['sessions_two_areas']:
    mice_sess[session[0:6]].append(session[8:])


In [36]:
# loop over mice here and store

subj = list(mice_sess.keys())[0]

ses_files = os.listdir(os.path.join(data_path,f'sub-{subj}'))

# get lfp files and spike files
lfp_files = list(filter(lambda s: 'None' in s, ses_files))
session = mice_sess['subj']
ses_file = list(filter(lambda s: session in s, ses_files))

layer_lfp = get_lfp_dict(subj, data_path, lfp_files, ses_file, toi, down_srate, roi)


